In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_data = requests.get(url).text
data = BeautifulSoup(raw_data, 'lxml')

In [13]:
cols = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = cols)

content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
postal_code,borough,neighborhood = 0,0,0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postal_code = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [27]:
df.head()

,Postalcode,Borough,Neighborhood
0,M9Z\n,Not assigned\n,Not assigned\n
1,M9Z\n,Not assigned\n,Not assigned\n
2,M9Z\n,North York\n,Parkwoods
3,M9Z\n,North York\n,Victoria Village
4,M9Z\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [15]:
df = df[df.Borough!='Not assigned']
df = df[df.Borough!= 0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1

In [17]:
df_new = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [25]:
df_new = df_new.dropna()

In [26]:
df_new.head()

,Postalcode,Borough,Neighborhood
0,M9Z\n,Central Toronto\n,"Lawrence Park, Roselawn, Davisville North, For..."
1,M9Z\n,Downtown Toronto\n,"Regent Park, Harbourfront, Queen's Park, Ontar..."
2,M9Z\n,East Toronto\n,"The Beaches, The Danforth West, Riverdale, Ind..."
3,M9Z\n,East York\n,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,M9Z\n,Etobicoke\n,"Islington Avenue, Humber Valley Village, West ..."


In [23]:
df_new = df_new[(df_new.Postalcode != 'Not assigned' ) & (df_new.Borough != 'Not assigned') & (df_new.Neighborhood != 'Not assigned')]

In [24]:
df_new.head()

,Postalcode,Borough,Neighborhood
0,M9Z\n,Central Toronto\n,"Lawrence Park, Roselawn, Davisville North, For..."
1,M9Z\n,Downtown Toronto\n,"Regent Park, Harbourfront, Queen's Park, Ontar..."
2,M9Z\n,East Toronto\n,"The Beaches, The Danforth West, Riverdale, Ind..."
3,M9Z\n,East York\n,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,M9Z\n,Etobicoke\n,"Islington Avenue, Humber Valley Village, West ..."


In [31]:
def list_by_neighbor(groups):    
    return ', '.join(sorted(groups['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_final = grp.apply(list_by_neighbor).reset_index(name='Neighborhood')

In [32]:
df_final

,Postalcode,Borough,Neighborhood
0,M9Z\n,Central Toronto\n,"Davisville, Davisville North, Forest Hill Nort..."
1,M9Z\n,Downtown Toronto\n,"Berczy Park, CN Tower, King and Spadina, Railw..."
2,M9Z\n,East Toronto\n,"Business reply mail Processing Centre, South C..."
3,M9Z\n,East York\n,"East Toronto, Broadview North (Old East York),..."
4,M9Z\n,Etobicoke\n,"Alderwood, Long Branch, Eringate, Bloordale Ga..."
5,M9Z\n,Mississauga\n,Canada Post Gateway Processing Centre
6,M9Z\n,North York\n,"Bathurst Manor, Wilson Heights, Downsview Nort..."
7,M9Z\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."
8,M9Z\n,Scarborough\n,"Agincourt, Birch Cliff, Cliffside West, Cedarb..."
9,M9Z\n,West Toronto\n,"Brockton, Parkdale Village, Exhibition Place, ..."


In [34]:
df_final.shape

(11, 3)